https://www.kaggle.com/uciml/breast-cancer-wisconsin-data

https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
%matplotlib inline

In [ ]:
pd.options.display.max_columns = None

In [ ]:
data = pd.read_csv('data.csv')
data.head()

In [ ]:
data.drop(columns=['Unnamed: 32','id'], inplace=True)
data.head()

In [ ]:
data.isna().sum()

`diagnosis` - целевая переменная

In [ ]:
sns.countplot(data['diagnosis'])
display(data['diagnosis'].value_counts())

In [ ]:
data.describe()

Посмотрим, как каждая из переменных в наборе данных соотносится с другой. На графике ниже также будут построены диаграммы распределения каждой из переменных

In [ ]:
sns.pairplot(data, hue='diagnosis')

На вскикду можно сказать, что в наборе данных много коррелирующих переменных

Теперь построим диаграммы рассеивания для целевой переменной

In [ ]:
sns.set(style="whitegrid", palette="muted")
# Запоминаем целевую переменную отдельно
target = data['diagnosis']
# Запоминаем данные отдельно
features = data.drop(columns=['diagnosis'])
# Приводим данные к стандартизированному виду
features = (features - features.mean()) / (features.std())

In [ ]:
len(features.columns)

In [ ]:
swarmplot_data = pd.concat([target, features.iloc[:,0:10]],axis=1)
swarmplot_data = pd.melt(swarmplot_data,
                         id_vars="diagnosis",
                         var_name="features",
                         value_name="value")
plt.figure(figsize=(12,12))
sns.swarmplot(x="features", y="value", hue="diagnosis", data=swarmplot_data)
plt.xticks(rotation=90)

In [ ]:
swarmplot_data = pd.concat([target, features.iloc[:,10:20]],axis=1)
swarmplot_data = pd.melt(swarmplot_data,
                         id_vars="diagnosis",
                         var_name="features",
                         value_name="value")
plt.figure(figsize=(12,12))
sns.swarmplot(x="features", y="value", hue="diagnosis", data=swarmplot_data)
plt.xticks(rotation=90)

In [ ]:
swarmplot_data = pd.concat([target, features.iloc[:,20:30]],axis=1)
swarmplot_data = pd.melt(swarmplot_data,
                         id_vars="diagnosis",
                         var_name="features",
                         value_name="value")
plt.figure(figsize=(12,12))
sns.swarmplot(x="features", y="value", hue="diagnosis", data=swarmplot_data)
plt.xticks(rotation=90)

По диаграммам рассеивания можно увидеть, что на некоторых переменных достаточно хорошо просматривается разделение на классы. Это замечание понадобится нам в будущем

In [ ]:
corr_matrix = data.corr()

In [ ]:
f,ax = plt.subplots(figsize=(20, 20))
sns.heatmap(corr_matrix, annot=True, linewidths=.5, fmt= '.1f',ax=ax)

Ниже приведены рассуждения, которыми я руководствовался при исключении переменных из датасета. Код закомментирован, чтобы не плодить лишние изображения.

В рассмотрение брались переменные с коэффициентом корреляции >= 0.8 (иногда делались и исключения).

Решение о том какие из сильно коррелирующих переменных оставлять, а какие удалять принимались на основе диаграмм рассеивания и диаграмм распределения (именно в таком порядке).

Переменная `radius_mean` сильно коррелирует с `perimeter_mean` и `area_mean`; оставляем `area_mean`.

Переменная `texture_mean` сильно коррелирует с `texture_worst`; оставляем `texture_mean`.

Переменная `area_mean` сильно коррелирует с `area_worst`; оставляем `area_mean`.

Переменная `radius_se` сильно коррелирует с `area_se`, `perimeter_se`; оставляем `area_se`.

Переменная `radius_worst` сильно коррелирует с `area_worst`, `perimeter_worst`; оставляем `area_worst`.

In [ ]:
# drop_cols = ['radius_mean',
#              'perimeter_mean',
#              'texture_worst',
#              'radius_se',
#              'perimeter_se',
#              'radius_worst',
#              'perimeter_worst']
# corr_matrix = data.drop(columns=drop_cols).corr()
# f,ax = plt.subplots(figsize=(20, 20))
# sns.heatmap(corr_matrix, annot=True, linewidths=.5, fmt= '.1f',ax=ax)

Переменная `compactness_mean` сильно коррелирует с `concavity_mean`, `concave points_mean`; оставляем `concave points_mean`.

Переменная `compactness_worst` сильно коррелирует с `concavity_worst`, `concave points_worst`. Оставим `concave points_worst`.

In [ ]:
# drop_cols = ['radius_mean',
#              'perimeter_mean',
#              'compactness_mean',
#              'concavity_mean',
#              'texture_worst',
#              'radius_se',
#              'perimeter_se',
#              'radius_worst',
#              'compactness_worst',
#              'concavity_worst',
#              'perimeter_worst']
# corr_matrix = data.drop(columns=drop_cols).corr()
# f,ax = plt.subplots(figsize=(20, 20))
# sns.heatmap(corr_matrix, annot=True, linewidths=.5, fmt= '.1f',ax=ax)

Переменная `area_mean` сильно коррелирует с `area_worst`, `concave points_mean`, `concavity_points_worst`; оставляем `area_mean`.

Переменная `compactness_se` сильно коррелирует с `concavity_se`, `concave points_se`; оставляем `concavity_se`.

На этом отбор переменных закончился, строим финальную матрицу корелляций, чтобы убедиться, что все более-менее в порядке.

In [ ]:
drop_cols = ['radius_mean',
             'perimeter_mean',
             'compactness_mean',
             'concavity_mean',
             'concave points_mean',
             'area_worst',
             'concave points_worst',
             'texture_worst',
             'radius_se',
             'compactness_se',
             'concave points_se',
             'perimeter_se',
             'radius_worst',
             'compactness_worst',
             'concavity_worst',
             'perimeter_worst']
corr_matrix = data.drop(columns=drop_cols).corr()
f,ax = plt.subplots(figsize=(20, 20))
sns.heatmap(corr_matrix, annot=True, linewidths=.5, fmt= '.1f',ax=ax)

Теперь можно приступать к этапу пострения модели